In [1]:
import pandas as pd
from covalent_api import *
from config import API_KEY

In [2]:
pool_df = get_uniswapv3_pools(api_key=API_KEY, page_size=6000)
pool_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5680 entries, 0 to 5679
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   dex_name                        5680 non-null   object 
 1   chain_name                      5680 non-null   object 
 2   chain_id                        5680 non-null   object 
 3   pool_contract_address           5680 non-null   object 
 4   pool_fee                        5680 non-null   object 
 5   pool_tick_spacing               5680 non-null   object 
 6   pool_deploy_time                5680 non-null   object 
 7   contract_address_token_0        5680 non-null   object 
 8   contract_name_token_0           5674 non-null   object 
 9   contract_ticker_symbol_token_0  5674 non-null   object 
 10  contract_decimals_token_0       5677 non-null   float64
 11  logo_url_token_0                5680 non-null   object 
 12  supports_erc_token_0            56

In [3]:
# don't want to make requests for pools without valid tokens
pool_df.dropna(subset=['contract_ticker_symbol_token_0', 'contract_ticker_symbol_token_1'],
               axis=0, inplace=True)

stable_coins = [
    'USDT',
    'USDC',
    'DAI',
    'FRAX'
]
coins = [
    'WBTC',  # wrapped BTC
    'WETH',  # wrapped ETH
]

# filter for pools where one token is a stable coin and the other is in
# the coin list (above)
df1 = pool_df.loc[
    pool_df['contract_ticker_symbol_token_0'].isin(stable_coins) & 
    pool_df['contract_ticker_symbol_token_1'].isin(coins)
]
df2 = pool_df.loc[
    pool_df['contract_ticker_symbol_token_0'].isin(coins) & 
    pool_df['contract_ticker_symbol_token_1'].isin(stable_coins)
]
pool_df = pd.concat([df1, df2], axis=0, ignore_index=True)

In [7]:
%%time
# making 40+ requests is pretty slow unfortunately
# so this cell takes >5 minutes to run
for i, row in pool_df.iterrows():
    try:
        n = get_uniswapv3_counts(api_key=API_KEY, transaction='swaps', 
                                 pool_address=row['pool_contract_address'])         
    except (APIError, requests.ConnectionError):
        n = np.nan
    pool_df.at[i, 'n_swaps'] = n

    try:
        n = get_uniswapv3_counts(api_key=API_KEY, transaction='liquidity', 
                                 pool_address=row['pool_contract_address'])  
    except (APIError, requests.ConnectionError):
        n = np.nan
    pool_df.at[i, 'n_liquidity'] = n

CPU times: user 903 ms, sys: 93.4 ms, total: 997 ms
Wall time: 6min 4s


In [8]:
# view the main results
cols = [
    'pool_contract_address',
    'contract_ticker_symbol_token_0',
    'contract_ticker_symbol_token_1',
    'n_swaps',
    'n_liquidity',
    'pool_deploy_time',
    'pool_fee'
]
pool_df.loc[:, cols].sort_values('n_swaps', ascending=False)

,pool_contract_address,contract_ticker_symbol_token_0,contract_ticker_symbol_token_1,n_swaps,n_liquidity,pool_deploy_time,pool_fee
6,0x60594a405d53811d3bc4766596efd80fd545a270,DAI,WETH,295146.0,3506.0,2021-05-05 18:03:43+00:00,500
7,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,USDC,WETH,193222.0,79595.0,2021-05-04 23:10:00+00:00,3000
21,0x4e68ccd3e89f51c3074ca5072bbac773960dfa36,WETH,USDT,158414.0,29214.0,2021-05-05 16:37:08+00:00,3000
8,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,DAI,WETH,84479.0,11014.0,2021-05-04 20:16:08+00:00,3000
20,0x99ac8ca7087fa4a2a1fb6357269965a2014abc35,WBTC,USDC,59411.0,10896.0,2021-05-05 19:13:15+00:00,3000
19,0x9db9e0e53058c89e5b94e29621a205198648425b,WBTC,USDT,23275.0,814.0,2021-05-05 19:21:53+00:00,3000
15,0x9a772018fbd77fcd2d25657e5c547baff3fd7d16,WBTC,USDC,21196.0,118.0,2021-06-03 12:41:15+00:00,500
9,0x7bea39867e4169dbe237d55c8242a8f2fcdcc387,USDC,WETH,16819.0,1997.0,2021-05-04 20:07:45+00:00,10000
17,0xc5af84701f98fa483ece78af83f11b6c38aca71d,WETH,USDT,13158.0,1012.0,2021-05-05 19:50:24+00:00,10000
5,0xa80964c5bbd1a0e95777094420555fead1a26c1e,DAI,WETH,5774.0,527.0,2021-05-05 19:06:59+00:00,10000


The USDC/ETH pool (0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640) is actually the largest pool, so it is a little concerning that we're encountering errors when trying to query the data. This was working yesterday (1/25), so hopefully this is transitory and we are able to pull the data eventually.

In [9]:
# update the data types before outputting
dtypes = {
    'pool_fee': int,
    'pool_tick_spacing': int,
    'contract_decimals_token_0': int,
    'contract_decimals_token_1': int
}
pool_df = pool_df.astype(dtypes)
pool_df['pool_deploy_time'] = pd.to_datetime(pool_df['pool_deploy_time'])
pool_df.reset_index(drop=True, inplace=True)

pool_df.to_csv('./data/pool_info.csv')